##### This notebook implements an updated version of the game of networks. 
##### The first version (match.py) is played between node pairs and a strategy is used to select a "card" from the CVS "deck".
##### The first version had several problems.
#####    - The strategy implementation (av_choice, mm_choice) was a derivative of the original strategy (average, MM)
#####    - The turns were repeats for all strategies, except the random strategy
##### In this game, the players are random groups of nodes, which makes it the game of neworks. The size of a random subnet is a fraction of the total node number (implanted iEEG channels; e.g., 0.1X). The number of players is also dependant on the total node number (e.g., 5X). 
##### A player's deck consisted of each pair's connectivity change score. The score is a function of the cross-validation scores (CVS). We considered several options:
#####    - Coefficient of variation
#####    - MM
#####    - Average
#####    - Max
#####    - Random
##### All players draw a card simultaneously, without knowing the other player's cards (incomplete information game).
##### The card selection is RANDOM. Why? Because the game is NOT a simulation of the connectivity dynamics that happend under the hood of the EEG signal. The game is an analysis tool, that merely identifies the groups that change their connectivity the most. 
##### The game finishes once the decks are exhausted.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint, sample
from scipy.interpolate import interp1d
from pickle import dump, load

woi = "preseizure1"

# from google.colab import drive
# drive.mount('/content/gdrive')
# main_folder = "/content/gdrive/My Drive/epigame-folder/"

# main_folder = "/media/kivi/ADATA HV100/epigame-folder/"
# main_folder = "/media/kivi/ADATA HV100/epigame-folder/epigame-folder-20230926T081603Z-023/epigame-folder/30sec/"
main_folder = "/home/kivi/Documents/"

# df = pd.read_csv(main_folder + f"cvs_pairs_{woi}.csv")

In [ ]:
df_beta = pd.read_csv(main_folder + f"cvs_pairs_{woi}.csv")
df_gamma = pd.read_csv(main_folder + f"cvs_pairs_{woi}_gamma.csv")

df = pd.concat([df_beta, df_gamma], ignore_index=True)

df.columns
# Exclude rows with "PEC" in "CM" column
# df = df[df['CM'] != 'PEC']

In [ ]:
connectivity_measures = list(df['CM'].unique())
connectivity_measures.remove("PEC")
print(connectivity_measures)

In [ ]:
NODES = {
"1":['P1-P2', 'P4-P5', 'P8-P9', 'P9-P10', 'P10-P11', 'G1-G2', 'G8-G9', 'G9-G10', 'G10-G11', 'G11-G12', 'M1-M2', 'M8-M9', 'M9-M10', 'M10-M11', 'M11-M12', 'O1-O2', 'O2-O3', 'O5-O6', 'O6-O7', 'F1-F2', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'F11-F12', 'F12-F13', 'A1-A2', 'A2-A3', 'A3-A4', 'A7-A8', 'A8-A9', 'A9-A10', 'A10-A11', 'B1-B2', 'B2-B3', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'C1-C2', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q4-Q5', 'Q8-Q9', 'Q9-Q10', 'Q10-Q11', 'Q11-Q12', 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'T9-T10', 'T10-T11', 'T11-T12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'E8-E9', 'E9-E10', 'E10-E11', 'L1-L2', 'L2-L3', 'L5-L6', 'L6-L7', 'L7-L8', 'U1-U2', 'U2-U3', 'U3-U4', 'U4-U5', 'U5-U6', 'U6-U7', 'J1-J2', 'J9-J10', 'J10-J11', 'J11-J12', 'J12-J13', 'J13-J14', 'J14-J15'],
"2":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8'],
"3":["T'1-T'2", "T'2-T'3", "T'3-T'4", "T'4-T'5", "T'5-T'6", "T'6-T'7", "T'7-T'8", "A'1-A'2", "A'2-A'3", "A'3-A'4", "A'4-A'5", "A'5-A'6", "A'6-A'7", "A'7-A'8", "A'8-A'9", "A'9-A'10", "B'1-B'2", "B'2-B'3", "B'3-B'4", "B'4-B'5", "B'5-B'6", "B'6-B'7", "B'7-B'8", "B'8-B'9", "B'9-B'10", "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10", "C'10-C'11", "C'11-C'12", "E'1-E'2", "E'2-E'3", "E'3-E'4", "E'4-E'5", "E'5-E'6", "E'6-E'7", "E'7-E'8", "E'8-E'9", "E'9-E'10", "D'1-D'2", "D'2-D'3", "D'3-D'4", "D'4-D'5", "D'5-D'6", "D'6-D'7", "D'7-D'8", "D'8-D'9", "D'9-D'10", "D'10-D'11", "D'11-D'12", "W'1-W'2", "W'2-W'3", "W'3-W'4", "W'4-W'5", "W'5-W'6", "W'6-W'7", "W'7-W'8", "W'8-W'9", "W'9-W'10", "W'10-W'11", "W'11-W'12", "W'12-W'13", "W'13-W'14", "W'14-W'15", "K'1-K'2", "K'2-K'3", "K'3-K'4", "K'4-K'5", "K'5-K'6", "K'6-K'7", "K'7-K'8", "K'8-K'9", "K'9-K'10", "K'10-K'11", "K'11-K'12", "K'12-K'13", "K'13-K'14", "K'14-K'15", "P'1-P'2", "P'2-P'3", "P'3-P'4", "P'4-P'5", "P'5-P'6", "P'6-P'7", "P'7-P'8", "P'8-P'9", "P'9-P'10", "P'10-P'11", "P'11-P'12", "P'12-P'13", "P'13-P'14", "P'14-P'15", "L'1-L'2", "L'2-L'3", "L'3-L'4", "L'4-L'5", "L'5-L'6", "L'6-L'7", "L'7-L'8", "L'8-L'9", "L'9-L'10", "L'10-L'11", "L'11-L'12", "O'1-O'2", "O'2-O'3", "O'3-O'4", "O'4-O'5", "O'5-O'6", "O'6-O'7", "O'7-O'8", "O'8-O'9", "O'9-O'10", "O'10-O'11", "O'11-O'12", "X'1-X'2", "X'2-X'3", "X'3-X'4", "X'4-X'5", "X'5-X'6", "X'6-X'7", "X'7-X'8", "X'8-X'9", "X'9-X'10", "X'10-X'11", "X'11-X'12", "X'12-X'13", "X'13-X'14", "X'14-X'15"],
"4":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'A8-A9', 'A9-A10', 'A10-A11', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B11-B12', 'C1-C2', 'C2-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'R1-R2', 'R2-R3', 'R3-R4', 'R4-R5', 'R5-R6', 'R6-R7', 'R7-R8', 'R8-R9', 'R9-R10', 'R10-R11', 'R11-R12', 'R12-R13', 'L1-L2', 'L2-L3', 'L3-L4', 'L4-L5', 'L5-L6', 'L6-L7', 'L7-L8', 'L8-L9', 'L9-L10', 'O1-O2', 'O2-O3', 'O3-O4', 'O4-O5', 'O5-O6', 'O6-O7', 'O7-O8', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q4-Q5', 'Q5-Q6', 'Q6-Q7', 'Q7-Q8', 'Q8-Q9', 'Q9-Q10', 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'T9-T10', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D6', 'D6-D7', 'D7-D8', 'D8-D9', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E8', 'E8-E9', 'E9-E10', 'E10-E11', 'J1-J2', 'J2-J3', 'J6-J7', 'J7-J8', 'J8-J9', 'J9-J10', 'J10-J11', 'J11-J12', 'I1-I2', 'I2-I3', 'I3-I4', 'I4-I5', 'I5-I6', 'I6-I7', 'I7-I8', 'P4-P5', 'P5-P6', 'P6-P7', 'P7-P8', 'P8-P9', 'P9-P10'],
"5":["A'1-A'2", "A'2-A'3", "A'3-A'4", "A'4-A'5", "A'5-A'6", "A'6-A'7", "A'7-A'8", "A'8-A'9", "A'9-A'10", "B'1-B'2", "B'2-B'3", "B'3-B'4", "B'4-B'5", "B'5-B'6", "B'6-B'7", "B'7-B'8", "B'8-B'9", "B'9-B'10", "B'10-B'11", "B'11-B'12", "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10", "C'10-C'11", "C'11-C'12", "D'1-D'2", "D'2-D'3", "D'3-D'4", "D'4-D'5", "D'5-D'6", "D'6-D'7", "D'7-D'8", "F'1-F'2", "F'2-F'3", "F'3-F'4", "F'4-F'5", "F'5-F'6", "F'6-F'7", "F'7-F'8", "H'1-H'2", "H'2-H'3", "H'3-H'4", "H'4-H'5", "G'1-G'2", "G'2-G'3", "G'3-G'4", "G'4-G'5", "G'5-G'6", "G'6-G'7", "G'7-G'8", "G'8-G'9", "G'9-G'10", "G'10-G'11", "G'11-G'12"],
"6":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'B11-B12', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'D8-D9', 'D9-D10', 'D10-D11', 'D11-D12', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'E8-E9', 'E9-E10', 'E10-E11', 'E11-E12', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'F11-F12', 'G1-G2', 'G2-G3', 'G3-G4', 'G4-G5', 'H1-H2', 'H2-H3', 'H3-H4', 'H4-H5', 'H5-H6', 'H6-H7', 'H7-H8', 'H8-H9', 'H9-H10', 'H10-H11', 'H11-H12', 'I1-I2', 'I2-I3', 'I3-I4', 'I4-I5', 'I5-I6', 'I6-I7', 'I7-I8', 'I8-I9', 'I9-I10', 'I10-I11', 'I11-I12', "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10"],
"7":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'B11-B12', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10", "C'10-C'11", "C'11-C'12", 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'E8-E9', 'E9-E10', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'F11-F12', 'G1-G2', 'G2-G3', 'G3-G4', 'G4-G5', 'G5-G6', 'G6-G7', 'G7-G8', 'G8-G9', 'G9-G10', 'H1-H2', 'H2-H3', 'H3-H4', 'H4-H5', 'H5-H6', 'H6-H7', 'H7-H8', 'I1-I2', 'I2-I3', 'I3-I4', 'I4-I5', 'I5-I6', 'I6-I7', 'I7-I8', 'I8-I9', 'I9-I10', 'I10-I11', 'I11-I12', 'J1-J2', 'J2-J3', 'J3-J4', 'J4-J5', 'J5-J6', 'J6-J7', 'J7-J8'],
"8":["F'1-F'2", "F'2-F'3", "F'8-F'9", "F'9-F'10", "F'10-F'11", "F'11-F'12", "T'1-T'2", "T'2-T'3", "T'3-T'4", "T'4-T'5", "T'5-T'6", "T'6-T'7", "T'7-T'8", "T'8-T'9", "T'9-T'10", "A'1-A'2", "A'2-A'3", "A'3-A'4", "A'4-A'5", "A'5-A'6", "A'6-A'7", "A'7-A'8", "A'8-A'9", "A'9-A'10", "A'10-A'11", "A'11-A'12", "B'1-B'2", "B'2-B'3", "B'3-B'4", "B'4-B'5", "B'8-B'9", "B'9-B'10", "B'10-B'11", "B'11-B'12", "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10", "C'10-C'11", "D'1-D'2", "D'2-D'3", "D'3-D'4", "D'4-D'5", "D'5-D'6", "D'6-D'7", "D'7-D'8", "D'8-D'9", "D'9-D'10", "S'1-S'2", "S'2-S'3", "S'5-S'6", "S'6-S'7", "S'7-S'8", "S'8-S'9", "S'9-S'10", "S'10-S'11", "S'11-S'12", "S'12-S'13", "S'13-S'14", "I'1-I'2", "I'2-I'3", "I'3-I'4", "I'4-I'5", "I'5-I'6", "I'6-I'7", "I'7-I'8", "I'8-I'9", "I'9-I'10", "W'1-W'2", "W'2-W'3", "W'10-W'11", "W'11-W'12", "W'12-W'13", "W'13-W'14", "W'14-W'15", "W'15-W'16", "W'16-W'17", "W'17-W'18", "U'1-U'2", "U'2-U'3", "U'3-U'4", "U'4-U'5", "U'8-U'9", "U'9-U'10", "U'10-U'11", "U'11-U'12", "U'12-U'13", "U'13-U'14", "U'14-U'15", "O'1-O'2", "O'2-O'3", "O'3-O'4", "O'4-O'5", "O'5-O'6", "O'6-O'7", "O'7-O'8", "O'8-O'9", "O'9-O'10", "O'10-O'11", "O'11-O'12", "O'12-O'13", "O'13-O'14", "O'14-O'15"],
"9":['FC1-FC2', 'FC2-FC3', 'FC3-FC4', 'FC4-FC5', 'FC5-FC6', 'FC6-FC7', 'FC7-FC8', 'FC8-FC9', 'FC9-FC10', 'A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'A8-A9', 'A9-A10', 'HAn1-HAn2', 'HAn2-HAnt3', 'HAnt3-HAnt4', 'HAnt4-HAnt5', 'HAnt5-HAnt6', 'HAnt6-HAnt7', 'HAnt7-HAnt8', 'HAnt8-HAnt9', 'HAnt9-HAnt10', 'HAnt10-Ref', 'HAnt11-Ref', 'HP1-HP2', 'HP2-HP3', 'HP3-HP4', 'HP4-HP5', 'HP5-HP6', 'HP6-HP7', 'HP7-HP8', 'HP8-HP9', 'HP9-HP10', 'TB1-TB2', 'TB2-TB3', 'TB3-TB4', 'TB4-TB5', 'TB5-TB6', 'TB6-TB7', 'TB7-TB8', 'TB8-TB9', 'TB9-TB10', 'TB10-TB11', 'TB11-TB12'],
"10":['F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'F11-F12', 'F12-F13', 'F13-F14', 'F14-F15', 'F15-F16', 'F16-F17', 'A1-A3', 'A3-A5', 'A5-A7', 'A7-A9', 'A9-A11', 'A11-A13', 'A13-A15', 'B1-B3', 'B3-B5', 'B5-B7', 'B7-B9', 'C1-C3', 'C3-C5', 'C5-C7', 'C7-C9', 'C9-C11', 'D1-D3', 'D3-D5', 'D5-D7', 'D7-D9', 'D9-D11', 'D11-D13', 'D13-D15', 'E1-E3', 'E3-E5', 'E5-E7', 'E7-E9', 'E9-E11', 'J1-J3', 'J3-J5', 'J5-J7', 'J7-J9', 'J9-J11', 'J11-J13', 'J13-J15', 'J15-J17', 'K1-K3', 'K3-K5', 'K5-K7', 'K7-K9', 'K9-K11', 'K11-K13', 'K13-K15', 'L1-L3', 'L3-L5', 'L5-L7', 'L7-L9', 'L9-L11', 'L11-L13', 'L13-L15', 'M1-M3', 'M3-M5', 'M5-M7', 'M7-M9', 'O1-O3', 'O3-O5', 'O5-O7', 'P1-P3', 'P3-P5', 'P5-P7', 'P7-P9', 'R1-R3', 'R3-R5', 'R5-R7', 'R7-R9', 'R9-R11', 'R11-R13', 'R13-R15', 'S1-S3', 'S3-S5', 'S5-S7', 'S7-S9', 'S9-S11', 'S11-S13', 'S13-S15', 'T1-T3', 'T3-T5', 'T5-T9', 'T9-T11'],
"11":["R'1-R'2", "R'2-R'3", "R'9-R'10", "R'10-R'11", "R'11-R'12", "R'12-R'13", "R'13-R'14", "S'1-S'2", "S'2-S'3", "S'6-S'7", "P'1-P'2", "P'2-P'3", "P'3-P'4", "P'4-P'5", "P'7-P'8", "P'8-P'9", "M'1-M'2", "M'8-M'9", "M'9-M'10", "M'10-M'11", "M'13-M'14", "J'1-J'2", "J'7-J'8", "J'8-J'9", "J'9-J'10", "K'1-K'2", "K'2-K'3", "K'7-K'8", "K'8-K'9", "K'9-K'10", "L'2-L'3", "L'3-L'4", "L'4-L'5", "L'5-L'6", "B'1-B'2", "B'5-B'6", "B'6-B'7", "B'7-B'8", "B'8-B'9", "C'1-C'2", "C'2-C'3", "C'9-C'10", "C'10-C'11", "C'11-C'12", "O'4-O'5", "O'5-O'6", "O'6-O'7", "O'10-O'11", "O'11-O'12", "O'12-O'13", "Q'3-Q'4", "Q'4-Q'5", "Q'5-Q'6", "Q'6-Q'7", "Q'10-Q'11", "Q'11-Q'12", 'S1-S2', 'S8-S9', 'S9-S10', 'S10-S11', 'P1-P2', 'P2-P3', 'P6-P7', 'P7-P8', 'K2-K3', 'K3-K4', 'K4-K5', 'L1-L2', 'L2-L3', 'L3-L4', 'L7-L8', 'J1-J2', 'J11-J12', 'J12-J13', 'J13-J14', 'C1-C2', 'C2-C3', 'C8-C9', 'C9-C10', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q12-Q13', 'Q13-Q14', 'O1-O2', 'O2-O3', 'O3-O4', 'O4-O5', 'O9-O10'],
"12":["E'1-E'2", "E'2-E'3", "E'3-E'4", "E'4-E'5", "E'5-E'6", "E'6-E'7", "E'7-E'8", "E'10-E'11", "O'1-O'2", "O'2-O'3", "O'3-O'4", "O'4-O'5", "O'5-O'6", "O'6-O'7", "O'7-O'8", "O'8-O'9", "O'9-O'10", "O'10-O'11", "C'1-C'2", "C'2-C'3", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10", "C'10-C'11", "T'1-T'2", "T'2-T'3", "T'8-T'9", "T'9-T'10", "T'10-T'11", "U'1-U'2", "U'4-U'5", "U'5-U'6", "U'8-U'9", "Q'1-Q'2", "Q'10-Q'11", "Q'11-Q'12", "Q'12-Q'13", "Q'13-Q'14", "Q'14-Q'15", "W'1-W'2", "W'6-W'7", "W'7-W'8", "W'8-W'9", "W'12-W'13", "W'13-W'14", "S'1-S'2", "S'8-S'9", "S'9-S'10", "S'10-S'11", "S'11-S'12", "P'1-P'2", "P'2-P'3", "P'3-P'4", "P'4-P'5", "P'5-P'6", "P'6-P'7", "P'7-P'8", "P'8-P'9", 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T11-T12', 'O1-O2', 'O2-O3', 'O5-O6', 'O6-O7', 'O7-O8', 'O8-O9', 'O9-O10', 'O10-O11', 'U1-U2', 'U5-U6', 'U6-U7', 'W1-W2', 'W2-W3', 'W3-W4', 'W9-W10', 'W10-W11', 'W11-W12', 'W12-W13', 'Q1-Q2', 'Q2-Q3', 'Q6-Q7', 'Q7-Q8', 'Q13-Q14', 'Q14-Q15', 'P1-P2', 'P4-P5', 'P5-P6', 'P6-P7', 'P10-P11'],
"13":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'A8-A9', 'A9-A10', 'A10-A11', 'A11-A12', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'B11-B12', 'B12-B13', 'B13-B14', 'B14-B15', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'F11-F12', 'G1-G2', 'G2-G3', 'G3-G4', 'G4-G5', 'G5-G6', 'G6-G7', 'G7-G8', 'H1-H2', 'H2-H3', 'H3-H4', 'H4-H5', 'H5-H6', 'H6-H7', 'H7-H8', 'H8-H9', 'H9-H10', 'H10-H11', 'H11-H12', 'I1-I2', 'I2-I3', 'I3-I4', 'I4-I5', 'I5-I6', 'I6-I7', 'I7-I8', 'I8-I9', 'I9-I10', 'L1-L2', 'L2-L3', 'L3-L4', 'L4-L5', 'L5-L6', 'L6-L7', 'L7-L8'],
"14":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'D8-D9', 'D9-D10', 'D10-D11', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'E8-E9', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'F8-F9', 'F9-F10', 'G1-G2', 'G2-G3', 'G3-G4', 'G4-G5', 'H1-H2', 'H2-H3', 'H3-H4', 'H4-H5', 'H5-H6', 'H6-H7', 'H7-H8', "G'1-G'2", "G'2-G'3", "G'3-G'4", "G'4-G'5", "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10"],
"15":["A'1-A'2", "A'2-A'3", "A'6-A'7", "A'7-A'8", "B'1-B'2", "B'2-B'3", "B'3-B'4", "B'4-B'5", "B'5-B'6", "B'6-B'7", "B'7-B'8", "C'1-C'2", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10", "E'1-E'2", "E'2-E'3", "E'5-E'6", "E'6-E'7", "E'7-E'8", "E'8-E'9", "T'1-T'2", "T'2-T'3", "T'3-T'4", "T'8-T'9", "T'9-T'10", "T'10-T'11", "T'11-T'12", "T'12-T'13", "T'13-T'14", "T'14-T'15", "J'1-J'2", "J'2-J'3", "J'3-J'4", "J'4-J'5", "J'12-J'13", "J'13-J'14", "J'14-J'15", "I'1-I'2", "I'2-I'3", "I'3-I'4", "I'8-I'9", "I'9-I'10", "I'10-I'11", "O'1-O'2", "O'6-O'7", "O'7-O'8", "O'8-O'9", "O'9-O'10", "O'10-O'11", "G'1-G'2", "G'2-G'3", "G'10-G'11", "G'11-G'12", "G'12-G'13", "G'13-G'14", "G'14-G'15", "Q'1-Q'2", "Q'6-Q'7", "Q'7-Q'8", "Q'8-Q'9", "Q'11-Q'12", "Q'12-Q'13", "P'1-P'2", "P'2-P'3", "P'3-P'4", "P'4-P'5", "P'5-P'6", "P'6-P'7", "P'7-P'8", "P'8-P'9", "P'9-P'10", "U'1-U'2", "U'5-U'6", "U'6-U'7", "U'7-U'8", "U'8-U'9", "M'1-M'2", "M'2-M'3", "M'3-M'4", "M'4-M'5", "M'7-M'8", "M'8-M'9", "M'9-M'10", "M'10-M'11", "M'11-M'12", 'F1-F2', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'M1-M2', 'M2-M3', 'M3-M4', 'M12-M13', 'M13-M14'],
"16":["F'1-F'2", "F'5-F'6", "F'8-F'9", "F'9-F'10", "M'1-M'2", "M'8-M'9", "M'9-M'10", "M'10-M'11", "M'11-M'12", "M'12-M'13", "M'13-M'14", "O'4-O'5", "O'5-O'6", "O'6-O'7", "Y'1-Y'2", "Y'2-Y'3", "Y'3-Y'4", "Y'4-Y'5", "Y'5-Y'6", "Y'6-Y'7", "Y'13-Y'14", "Y'14-Y'15", "Y'15-Y'16", "Y'16-Y'17", "I'1-I'2", "I'5-I'6", "I'6-I'7", "I'7-I'8", "U'1-U'2", "U'2-U'3", "U'3-U'4", "U'4-U'5", "U'5-U'6", "A'1-A'2", "A'7-A'8", "A'8-A'9", "A'9-A'10", "A'10-A'11", "B'1-B'2", "B'2-B'3", "B'8-B'9", "B'9-B'10", "B'10-B'11", "C'1-C'2", "C'2-C'3", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'10-C'11", "C'11-C'12", "D'1-D'2", "D'6-D'7", "D'7-D'8", "D'8-D'9", "E'2-E'3", "E'3-E'4", "E'4-E'5", "E'5-E'6", "E'6-E'7", "E'7-E'8", "E'8-E'9", "E'9-E'10", "E'10-E'11", "J'1-J'2", "J'8-J'9", "J'9-J'10", "J'10-J'11", "J'11-J'12", "J'12-J'13", "J'13-J'14", "J'14-J'15", "W'1-W'2", "W'11-W'12", "W'12-W'13", "W'13-W'14", "Q'1-Q'2", "Q'2-Q'3", "Q'6-Q'7", "Q'7-Q'8", "Q'8-Q'9", "Q'9-Q'10", "Q'10-Q'11", 'A1-A2', 'A2-A3', 'A8-A9', 'A9-A10', 'A10-A11', 'B1-B2', 'B7-B8', 'B8-B9', 'C1-C2', 'C7-C8', 'C8-C9'],
"17":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'B11-B12', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'D8-D9', 'D9-D10', 'D10-D11', 'D11-D12', 'G1-G2', 'G2-G3', 'G3-G4', 'G4-G5', 'G5-G6', 'G6-G7', 'G7-G8', 'G8-G9', 'G9-G10', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'E8-E9', 'E9-E10', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'F8-F9', 'F9-F10', 'F10-F11', 'F11-F12', 'H1-H2', 'H2-H3', 'H3-H4', 'H4-H5', 'H5-H6', 'H6-H7', 'H7-H8', 'H8-H9', 'H9-H10', 'J1-J2', 'J2-J3', 'J3-J4', 'J4-J5', 'J5-J6', 'J6-J7', 'J7-J8', 'J8-J9', 'J9-J10', 'J10-J11', 'J11-J12', 'O1-O2', 'O2-O3', 'O3-O4', 'O4-O5', 'O5-O6', 'O6-O7', 'O7-O8', 'O8-O9', 'O9-O10', 'O10-O11', 'O11-O12', 'I1-I2', 'I2-I3', 'I3-I4', 'I4-I5', 'I5-I6', 'I6-I7', 'I7-I8', 'I8-I9', 'I9-I10', 'I10-I11', 'I11-I12', 'I12-I13', 'I13-I14', 'I14-I15', 'I15-I16', 'I16-I17', 'I17-I18'],
"18":['F1-F2', 'F2-F3', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F7-F8', 'S1-S2', 'S2-S3', 'S3-S4', 'S4-S5', 'S5-S6', 'S6-S7', 'S7-S8', 'S8-S9', 'S9-S10', 'S10-S11', 'S11-S12', 'H1-H2', 'H2-H3', 'H3-H4', 'H4-H5', 'H5-H6', 'H6-H7', 'H7-H8', 'M1-M2', 'M2-M3', 'M3-M4', 'M4-M5', 'M5-M6', 'M6-M7', 'M7-M8', 'U1-U2', 'U2-U3', 'U3-U4', 'U4-U5', 'U5-U6', 'U6-U7', 'U7-U8', 'A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'A8-A9', 'A9-A10', 'A10-A11', 'A11-A12', 'A12-A13', 'A13-A14', 'A14-A15', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'J1-J2', 'J2-J3', 'J3-J4', 'J4-J5', 'J5-J6', 'J6-J7', 'J7-J8', 'J8-J9', 'J9-J10', 'O1-O2', 'O2-O3', 'O3-O4', 'O4-O5', 'O5-O6', 'O6-O7', 'O7-O8', 'O8-O9', 'O9-O10', 'O10-O11', 'O11-O12', 'O12-O13', 'O13-O14', 'O14-O15', 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'T9-T10', 'L1-L2', 'L2-L3', 'L3-L4', 'L4-L5', 'L5-L6', 'L6-L7', 'L7-L8', 'L8-L9', 'L9-L10', 'P1-P2', 'P2-P3', 'P3-P4', 'P4-P5', 'P5-P6', 'P6-P7', 'P7-P8', 'P8-P9', 'P9-P10', 'P10-P11', 'P11-P12', 'P12-P13', 'P13-P14', 'P14-P15', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q4-Q5', 'Q5-Q6', 'Q6-Q7', 'Q7-Q8', 'Q8-Q9', 'Q9-Q10', 'Q10-Q11', 'Q11-Q12', 'Q12-Q13', 'Q13-Q14', 'Q14-Q15', 'X1-X2', 'X2-X3', 'X3-X4', 'X4-X5', 'X5-X6', 'X6-X7', 'X7-X8', 'X8-X9', 'X9-X10', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'Y1-Y2', 'Y2-Y3', 'Y3-Y4', 'Y4-Y5', 'Y5-Y6', 'Y6-Y7', 'Y7-Y8', 'Z1-Z2', 'Z2-Z3', 'Z3-Z4', 'Z4-Z5', 'Z5-Z6', 'Z6-Z7', 'Z7-Z8', 'Au1-Ref Au', 'Au2-Ref Au'],
"19":['A1-A2', 'A2-A3', 'A3-A4', 'A4-A5', 'A5-A6', 'A6-A7', 'A7-A8', 'B1-B2', 'B2-B3', 'B3-B4', 'B4-B5', 'B5-B6', 'B6-B7', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'B11-B12', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'D8-D9', 'D9-D10', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E5-E6', 'E6-E7', 'E7-E8', 'E8-E9', 'E9-E10', 'E10-E11', 'E11-E12', 'F1-F2', 'F2-F3', 'F3-F4', 'F4-F5'],
"20":['A1-A2', 'A2-A3', 'A6-A7', 'A7-A8', 'A8-A9', 'A10-A11', 'A11-A12', 'B1-B2', 'B2-B3', 'B3-B4', 'B7-B8', 'B8-B9', 'B9-B10', 'B10-B11', 'C1-C2', 'C2-C3', 'C3-C4', 'C4-C5', 'C5-C6', 'C6-C7', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'C11-C12', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D5-D6', 'D6-D7', 'D7-D8', 'D8-D9', 'D9-D10', 'D10-D11', 'D11-D12', 'J1-J2', 'J2-J3', 'J3-J4', 'J8-J9', 'J9-J10', 'J10-J11', 'J11-J12', 'J12-J13', 'J13-J14', 'J14-J15', 'K1-K2', 'K2-K3', 'K3-K4', 'K4-K5', 'K5-K6', 'K6-K7', 'K7-K8', 'K8-K9', 'K9-K10', 'K10-K11', 'K11-K12', 'K12-K13', 'K13-K14', 'P1-P2', 'P2-P3', 'P3-P4', 'P4-P5', 'P5-P6', 'P6-P7', 'P7-P8', 'P8-P9', 'P9-P10', 'P10-P11', 'P11-P12', 'O1-O2', 'O2-O3', 'O3-O4', 'O4-O5', 'O5-O6', 'O6-O7', 'O7-O8', 'F3-F4', 'F4-F5', 'F5-F6', 'F6-F7', 'F9-F10', 'F10-F11', 'F11-F12', "A'1-A'2", "A'2-A'3", "A'3-A'4", "A'4-A'5", "A'8-A'9", "A'9-A'10", "A'10-A'11", "A'11-A'12", "B'1-B'2", "B'2-B'3", "B'3-B'4", "B'7-B'8", "B'8-B'9", "B'9-B'10", "C'1-C'2", "C'2-C'3", "C'3-C'4", "C'4-C'5", "C'5-C'6", "C'6-C'7", "C'7-C'8", "C'8-C'9", "C'9-C'10"],
"21":['X1-X2', 'X2-X3', 'X3-X4', 'X4-X5', 'X5-X6', 'X6-X7', 'X7-X8', 'X10-X11', 'X11-X12', 'X12-X13', 'X13-X14', 'X14-X15', 'X15-X16', 'X16-X17', 'X17-X18', 'I1-I2', 'I4-I5', 'I5-I6', 'I8-I9', 'I9-I10', 'A1-A2', 'A2-A3', 'A3-A4', 'A7-A8', 'A8-A9', 'A9-A10', 'A10-A11', 'T1-T2', 'T2-T3', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'E1-E2', 'E2-E3', 'E3-E4', 'E4-E5', 'E6-E7', 'E7-E8', 'E8-E9', 'B1-B2', 'B2-B3', 'B3-B4', 'B8-B9', 'B9-B10', 'B10-B11', 'C1-C2', 'C2-C3', 'C3-C4', 'C7-C8', 'C8-C9', 'C9-C10', 'C10-C11', 'D1-D2', 'D2-D3', 'D3-D4', 'D4-D5', 'D8-D9', 'D9-D10', 'D10-D11', 'S1-S2', 'S2-S3', 'S3-S4', 'S4-S5', 'S12-S13', 'S13-S14', 'S14-S15', 'S15-S16', 'S16-S17', 'Q1-Q2', 'Q2-Q3', 'Q3-Q4', 'Q9-Q10', 'Q10-Q11', 'Q11-Q12', 'Q12-Q13', 'Q13-Q14', 'O1-O2', 'O2-O3', 'O3-O4', 'O4-O5', 'O5-O6', 'O6-O7', 'O7-O8', 'O8-O9', 'O9-O10', 'O10-O11', 'O11-O12', 'P1-P2', 'P2-P3', 'P3-P4', 'P4-P5', 'P5-P6', 'P11-P12', 'P12-P13', 'P13-P14', 'P14-P15']
}

RESECTION = {1: ['T9-T10', 'T4-T5', 'B6-B7', 'B2-B3', 'D4-D5', 'D2-D3', 'T6-T7', 'B1-B2', 'T10-T11', 'T11-T12', 'T3-T4', 'T8-T9', 'T7-T8', 'A3-A4', 'T1-T2', 'T5-T6', 'T2-T3', 'B5-B6', 'D3-D4'], 2: ['A1-A2', 'B2-B3', 'B6-B7', 'A2-A3', 'B1-B2', 'C2-C3', 'C7-C8', 'B8-B9', 'B4-B5', 'C9-C10', 'A3-A4', 'C3-C4', 'B7-B8', 'C1-C2', 'C5-C6', 'C8-C9', 'A4-A5', 'C6-C7', 'C4-C5', 'B5-B6', 'B9-B10', 'B3-B4'], 3: ["D'8-D'9", "D'11-D'12", "W'14-W'15", "P'12-P'13", "L'9-L'10", "L'11-L'12", "O'9-O'10", "O'10-O'11", "K'13-K'14", "O'11-O'12", "D'6-D'7", "L'10-L'11", "P'13-P'14", "D'9-D'10", "W'13-W'14", "P'14-P'15", "K'12-K'13", "D'7-D'8"], 4: ['A10-A11', 'P7-P8', 'P9-P10', 'O4-O5', 'Q8-Q9', 'E2-E3', 'E9-E10', 'P6-P7', 'L1-L2', 'E4-E5', 'L4-L5', 'Q7-Q8', 'J7-J8', 'C9-C10', 'J8-J9', 'O6-O7', 'P8-P9', 'C11-C12', 'E3-E4', 'I6-I7', 'J9-J10', 'B11-B12', 'L2-L3', 'O5-O6', 'O3-O4', 'J10-J11', 'Q3-Q4', 'C10-C11', 'E10-E11', 'P5-P6', 'J6-J7', 'J11-J12', 'J1-J2', 'Q4-Q5', 'I7-I8', 'A9-A10', 'L3-L4', 'Q9-Q10'], 5: ["D'4-D'5", "D'2-D'3", "D'3-D'4", "D'1-D'2", "B'1-B'2", "D'6-D'7", "D'5-D'6", "D'7-D'8"], 6: ['C1-C2', 'C4-C5', 'C2-C3', 'C3-C4'], 7: ['F11-F12', 'G5-G6', 'E1-E2', 'F8-F9', 'A1-A2', 'B2-B3', 'B6-B7', 'A2-A3', 'D4-D5', 'D2-D3', 'E2-E3', 'E9-E10', 'G7-G8', 'C2-C3', 'G9-G10', 'G2-G3', 'E4-E5', 'G6-G7', 'A5-A6', 'D1-D2', 'G8-G9', 'C7-C8', 'E8-E9', 'I8-I9', 'B8-B9', 'B4-B5', 'F7-F8', 'F9-F10', 'H7-H8', 'J7-J8', 'A7-A8', 'C9-C10', 'D5-D6', 'H4-H5', 'H5-H6', 'D7-D8', 'B10-B11', 'A3-A4', 'E6-E7', 'G1-G2', 'E3-E4', 'A6-A7', 'B7-B8', 'C1-C2', 'C3-C4', 'C5-C6', 'F10-F11', 'G3-G4', 'H2-H3', 'H3-H4', 'B11-B12', 'E5-E6', 'D6-D7', 'C8-C9', 'I11-I12', 'J6-J7', 'F6-F7', 'A4-A5', 'C6-C7', 'C4-C5', 'E7-E8', 'G4-G5', 'I7-I8', 'B5-B6', 'D3-D4', 'B9-B10', 'I9-I10', 'I10-I11', 'B3-B4', 'H6-H7'], 8: ["T'7-T'8", "D'2-D'3", "A'3-A'4", "T'6-T'7", "A'6-A'7", "D'4-D'5", "A'7-A'8", "T'2-T'3", "A'1-A'2", "A'5-A'6", "A'8-A'9", "D'5-D'6", "A'2-A'3", "D'8-D'9", "A'4-A'5", "T'5-T'6", "D'3-D'4", "T'4-T'5", "D'7-D'8", "T'1-T'2", "T'3-T'4", "D'1-D'2"], 9: ['A3-A4', 'TB5-TB6', 'A8-A9', 'TB4-TB5', 'A4-A5', 'A6-A7', 'TB6-TB7', 'A1-A2', 'A2-A3', 'A7-A8', 'TB2-TB3', 'A9-A10', 'TB1-TB2', 'A5-A6', 'TB3-TB4'], 10: ['F11-F12', 'F6-F7', 'F4-F5', 'F13-F14', 'F5-F6', 'F12-F13', 'F1-F2', 'F8-F9', 'F7-F8', 'O1-O3', 'F2-F3', 'T3-T5', 'O3-O5', 'F3-F4', 'T5-T9'], 11: ["O'5-O'6", "O'4-O'5", "M'1-M'2", "J'8-J'9", "Q'5-Q'6", "P'4-P'5", "K'9-K'10", "P'7-P'8", "P'1-P'2", "O'6-O'7", "Q'4-Q'5", "S'6-S'7", "K'8-K'9", "R'2-R'3", "J'7-J'8", "J'9-J'10", "P'8-P'9", "Q'3-Q'4", "K'1-K'2", "K'7-K'8", "J'1-J'2"], 12: ['O10-O11'], 13: ['F11-F12', 'F4-F5', 'I3-I4', 'F8-F9', 'B2-B3', 'B6-B7', 'F2-F3', 'I2-I3', 'B1-B2', 'C2-C3', 'C7-C8', 'I8-I9', 'B4-B5', 'F7-F8', 'F9-F10', 'H4-H5', 'H5-H6', 'I4-I5', 'F3-F4', 'I1-I2', 'I5-I6', 'F5-F6', 'C3-C4', 'B7-B8', 'H8-H9', 'I6-I7', 'C5-C6', 'F10-F11', 'H3-H4', 'C8-C9', 'F6-F7', 'C6-C7', 'C4-C5', 'I7-I8', 'B5-B6', 'H7-H8', 'B3-B4', 'H6-H7'], 14: ['E6-E7', 'E8-E9', 'E1-E2', 'E7-E8', 'E5-E6'], 15: ["J'13-J'14", "T'1-T'2", "J'2-J'3", "J'12-J'13", "T'2-T'3", "T'14-T'15", "T'13-T'14", "J'1-J'2", "J'14-J'15", "J'3-J'4"], 16: ["B'9-B'10", "D'8-D'9", "E'2-E'3", "A'7-A'8", "D'1-D'2", "B'1-B'2", "A'1-A'2", "A'10-A'11", "D'6-D'7", "A'8-A'9", "B'10-B'11", "A'9-A'10", "D'7-D'8"], 17: ['I3-I4', 'E1-E2', 'A1-A2', 'B2-B3', 'B6-B7', 'A2-A3', 'E2-E3', 'I2-I3', 'B1-B2', 'C2-C3', 'E4-E5', 'B8-B9', 'B4-B5', 'I4-I5', 'I1-I2', 'B10-B11', 'A3-A4', 'E6-E7', 'E3-E4', 'C3-C4', 'B7-B8', 'C1-C2', 'C5-C6', 'E5-E6', 'C6-C7', 'C4-C5', 'B5-B6', 'B9-B10', 'B3-B4'], 18: ['A10-A11', 'P7-P8', 'P9-P10', 'P4-P5', 'S3-S4', 'O13-O14', 'Q8-Q9', 'T4-T5', 'P6-P7', 'T6-T7', 'Y6-Y7', 'Q10-Q11', 'X5-X6', 'T7-T8', 'Q7-Q8', 'X4-X5', 'H4-H5', 'A7-A8', 'H5-H6', 'P8-P9', 'A8-A9', 'F5-F6', 'A6-A7', 'H2-H3', 'S9-S10', 'J2-J3', 'S5-S6', 'H3-H4', 'C3-C4', 'C5-C6', 'S10-S11', 'Y3-Y4', 'A11-A12', 'Y4-Y5', 'X3-X4', 'Q12-Q13', 'Q13-Q14', 'T5-T6', 'H1-H2', 'P5-P6', 'Y2-Y3', 'Y7-Y8', 'Y5-Y6', 'F6-F7', 'X6-X7', 'C6-C7', 'C4-C5', 'J1-J2', 'Q9-Q10', 'S4-S5', 'O12-O13', 'H7-H8', 'O11-O12', 'Q11-Q12', 'P3-P4', 'H6-H7'], 19: ['A1-A2', 'B2-B3', 'B6-B7', 'A2-A3', 'B1-B2', 'C2-C3', 'A5-A6', 'D1-D2', 'B8-B9', 'B4-B5', 'B10-B11', 'A3-A4', 'C3-C4', 'B7-B8', 'C1-C2', 'C5-C6', 'B11-B12', 'A4-A5', 'C4-C5', 'B5-B6', 'B9-B10', 'B3-B4'], 20: ['A6-A7', 'A1-A2', 'B2-B3', 'A2-A3', 'A7-A8', 'B1-B2', 'B3-B4'], 21: ['A10-A11', 'E1-E2', 'A1-A2', 'B2-B3', 'A2-A3', 'E2-E3', 'T6-T7', 'E4-E5', 'D1-D2', 'T8-T9', 'E8-E9', 'T7-T8', 'A7-A8', 'B10-B11', 'A3-A4', 'E6-E7', 'A8-A9', 'E3-E4', 'T1-T2', 'T5-T6', 'T2-T3', 'E7-E8', 'A9-A10', 'B3-B4']}

subs = list(range(1,22))
poor_outcome_subjects = [3,6,10,11,12,17,18]
good_outcome_subjects = [s for s in subs if s not in poor_outcome_subjects+[12]]


In [ ]:
# Set a specific random seed (e.g., 42 for reproducibility)
random_seed = 42
np.random.seed(random_seed)

In [ ]:
class Player:
    def __init__(self, AI, deck, name, n_in_hand=5):
        """
        Initializes a Player object.

        Parameters:
        - AI: The artificial intelligence strategy used by the player.
        - deck: The deck of cards available for the player.
        - name: The name of the player.
        - n_in_hand: The number of cards initially dealt to the player's hand.
        """
        self.deck = deck
        self.n_in_hand = n_in_hand
        self.logic = AI
        self.cards = []  # Player's hand
        self.score = []  # Player's score (not used in the provided code)
        self.name = name
        
    def shuffle(hand):
            hand.cards = sorted(hand.deck)

    def check(scores):
        """
        Parameters:
        - scores: The scores to be checked.

        Returns:
        - bool: True if the scores are real, otherwise False.
        """
        return scores.real

    def play(god, *players):
        """
        Determines the player's move in a game.

        Parameters:
        - god: The player making the move.
        - players: Other players in the game.
 
        Returns:
        - int: The chosen card to play.
        """
        best, other = max(god.cards), []
        for player in players:
            other += player.cards

        if best >= max(other):
            return best
        else:
            return min(god.cards)

    def card(draw):
        """
        Draws a card based on the player's AI logic.

        Parameters:
        - draw: The drawing logic.

        Returns:
        - int: The drawn card.
        """
        return draw.logic(draw)

def rn_choice(logic):
    return logic.cards.pop(randint(0,len(logic.cards)-1))

# # Example Usage:
# # Define an AI logic for drawing cards
# def draw_logic(draw):
#     # Replace this with your custom drawing logic
#     return randint(1, 10)

# # Create a deck of cards (assuming a list of integers)
# deck_of_cards = list(range(1, 11))

# # Create a player with the defined AI logic, deck, and name
# player1 = Player(AI=draw_logic, deck=deck_of_cards, name="Player1")


K-fold cross-validation scores summary statistics functions - AI logic in the game:

In [ ]:
def deck_average(of): return sum(of)/len(of)

def deck_mm(of): return min(of)*max(of)/np.mean(of)

def deck_coef_of_variation(of): return np.std(of)/np.mean(of)

def deck_max(of): return np.max(of)

def deck_random(of): return of[randint(0,len(of)-1)]

def pop_closest_card(cards, summary_stat):
    if not cards: return None
    # The min function then compares the tuples for each card, and it selects the card with the minimum tuple value. 
    # The first element of the tuple (abs(card - summary_stat)) is used as the primary key, and in case of a tie, the second element (-card) is used as a tiebreaker.
    closest_value = min(cards, key=lambda cards: (abs(cards - summary_stat), -cards))
    cards.remove(closest_value)
    return closest_value

###### The play function takes an arbitrary number of players (*game) as arguments.
###### 1. It initializes each player's real and best attributes to 0 and shuffles their cards.
###### 2. The resolve function is defined inside play. It takes a list of (player, card) pairs, where the card is drawn using the card method, and determines the winners.
###### 3. Winners are players with the highest card value. The function returns a list of winning players.
###### 4. The winners' real attribute is incremented.
###### 5. Each player's score list is updated by appending the result of the check method.

In [ ]:
def play(*game):
    # Initialize each player's real and best score attributes to 0
    for player in game:
        player.real, player.best = 0, 0
        player.shuffle()

    # Define a function 'resolve' to find the winners among the players
    def resolve(hand, by=[], best=-1):
        for player, card in hand:
            # Check if the card is equal to the best card, add player to 'by' list
            if best == card:
                by.append(player)
            # If the card is greater than the best, update 'best' and set 'by' to [player]
            if best < card:
                best = card
                by = [player]
        return by

    # Use the 'resolve' function to find the winners among the players
    winners = resolve([(player, player.card()) for player in game])

    # Increment the 'real' attribute for each winner
    for player in winners:
        player.real += 1

    # Update each player's score by appending the result of the 'check' method
    for player in game:
        player.score.append(player.check())


In [ ]:
from joblib import Parallel, delayed
from statannot import add_stat_annotation
import seaborn as sns
from collections import Counter

def check_winners(sorted_players, sigmas, resection, group_size):
    """
    Check winners' performance based on cross-validation scores and resection overlap.

    Parameters:
    - sorted_players (dict): Dictionary containing sorted players' cross-validation scores.
    - sigmas (float): Number of standard deviations for defining the threshold.
    - nodes (list): List of node labels.
    - resection (list): List of nodes in the resection area.
    - group_size (float): Player or random group node number.

    Returns:
    - Tuple: (number of winners above threshold, winner-loser ratio, median resection overlap, mean resection overlap).
    """

    # Calculate group scores and resection matches
    group_scores = {player:sum(sorted_players[player]) for player in sorted_players}
    resection_match = {player: [n for n in player if n in resection] for player in group_scores}

    # # Plot the wins distribution
    # FONT_SIZE = 13
    # count = dict(Counter(group_scores.values()))
    # count_sorted = {k: v for k, v in sorted(count.items(), key=lambda item: item[0])}
    # counted = {"N_nodes": list(count_sorted.values()), "Scores": list(count_sorted.keys())}
    # fig, ax = plt.subplots(1, 2, gridspec_kw={'width_ratios': [4, 2]}, figsize=(6, 3))
    # fig.tight_layout()
    # sns.barplot(data=counted, x="Scores", y="N_nodes", ax=ax[0], color="grey")
    # ax[0].set_xlabel("Game score (Number of wins)", fontsize=FONT_SIZE)
    # ax[0].set_xticklabels(counted["Scores"], rotation=90)
    # ax[0].set_ylabel("Number of players", fontsize=FONT_SIZE)
    # sns.boxplot(x=list(group_scores.values()), ax=ax[1], color="grey")
    # plt.show()

    # Calculate statistical measures
    median = np.median(list(group_scores.values()))
    mean = np.average(list(group_scores.values()))
    std = np.std(list(group_scores.values()))

    # Define threshold based on mean and standard deviations
    thresh = mean + (std * sigmas)
    winners_above_thresh = [players for players in group_scores if group_scores[players] >= thresh]
    N_winners_above_thresh = len(winners_above_thresh)
    print(f"Number of winners above threshold ({thresh}) = {N_winners_above_thresh}")

    if N_winners_above_thresh > 0:
        # Calculate resection overlap for winners and losers
        resection_overlap_winners = [len(resection_match[players]) / group_size for players in winners_above_thresh]
        resection_overlap_losers = [len(resection_match[players]) / group_size for players in resection_match if
                                    players not in winners_above_thresh]

        # Calculate the winner-loser ratio
        winner_loser_ratio = np.mean(resection_overlap_winners) / np.mean(resection_overlap_losers)
        print("Winner Resection Overlap/Loser Resection Overlap =", winner_loser_ratio)

        # # Plot resection overlap percentage
        # data = pd.DataFrame({"Resection overlap": resection_overlap_winners + resection_overlap_losers,
        #                      "Player outcome": list("Winner" for _ in range(len(resection_overlap_winners))) + list(
        #                          "Loser" for _ in range(len(resection_overlap_losers)))})
        # ax2 = sns.boxplot(data=data, x='Player outcome', y='Resection overlap', hue='Player outcome')
        # add_stat_annotation(ax2, data=data, x='Player outcome', y='Resection overlap', box_pairs=[("Winner", "Loser")],
        #                     test='Mann-Whitney', text_format='star', loc='outside', verbose=2)
        # plt.show()

        return N_winners_above_thresh, winner_loser_ratio, np.median(resection_overlap_winners), np.mean(resection_overlap_winners),

    else:
        return N_winners_above_thresh, 0, 0, 0


In [ ]:
# for testing
subs = [2]
connectivity_measures = ["PLI-(0,4)"]

In [ ]:
path_game_scores = "/media/kivi/ADATA HV100/epigame-folder/game_vKarla/"

summary_stat_label = ["average", "mm", "coefvar", "max", "random"]

Subject, Outcome, CM, R, Group_size, Strategy, Sigmas, N_winners, Mean_overlap_ratio, Median_overlap_winners, Mean_overlap_winners = [],[],[],[],[],[],[],[],[],[],[]

for sub in subs:

    surgical_outcome = 1 if sub in good_outcome_subjects else 0

    all_nodes = NODES[str(sub)]
    # print(f"Total nodes = {len(all_nodes)}")

    resection_nodes = RESECTION[sub]
    # print(f"Resection nodes = {len(resection_nodes)}")

    df_sub = df[df['Subject'] == sub]

    r = 0.1
    group_size = int(len(all_nodes)*r)
    # print("Random group size =", group_size)

    players = [sorted(list(np.random.choice(all_nodes, size=group_size, replace=False))) for i in range(len(all_nodes)*5)]

    for i,summary_stat in enumerate([deck_average, deck_mm, deck_coef_of_variation, deck_max, deck_random]):

        for measure in connectivity_measures:
            print(f"Subject: {sub}")
            print(f"Connectivity measure: {measure}")
            print(f"Summary statisctic for connectivity change: {summary_stat_label[i]}")

            df_sub_cm = df_sub[df_sub['CM'] == measure]

            hands = [] # initialize hands, a list of tuples (player, deck)

            for group_labels in players:
                if group_labels not in [hand[0] for hand in hands]:

                    # Split the Labels column into two nodes using '<->'
                    split_labels = df_sub_cm['Labels'].str.split('<->', expand=True)

                    # Check if both nodes are in group_labels
                    relevant_rows = df_sub_cm[((split_labels[0].isin(group_labels)) & split_labels[1].isin(group_labels))]
                    # Calculate (max(CVS)+min(CVS)) / mean(CVS) scores (MM scores)
                    group_deck = list(relevant_rows['CVS'].apply(lambda x: summary_stat(list(map(float, x.strip('[]').split())))))
                    hands.append((group_labels, group_deck))

            n_cards = len(hands[0][1]) # number of cards in each player's deck

            remap = interp1d([0,1],[1,100]) # remap values from 0 to 1, to 1 to 100
            
            game_score = {tuple(p):[] for p in [hand[0] for hand in hands]}

            rounds=10
            for turn in range (rounds):

                for r in range(n_cards):

                    game = [Player(AI=rn_choice, deck=hand[1], name=tuple(hand[0]), n_in_hand=n_cards) for hand in hands]

                    play(*game)

                    scores = sorted([(player.name, player.score) for player in game], key=lambda x:x[1], reverse=True)

                    top_score, fall = scores[0][1], 0
                    for name, score in scores:
                        if score==top_score: game_score[name].append(1); fall+=1
                        elif score!=top_score: break
                    for name,score in scores[fall:]: game_score[name].append(0)

            sorted_players = {k:v for k, v in sorted(game_score.items(), key=lambda item: sum(item[1]), reverse=True)}
            dump(sorted_players, open(path_game_scores + f"scores_{measure}_{summary_stat_label[i]}_{n_cards}cards_{rounds}rounds_preseizure1_sub{sub}.p", "wb"))

            sigmas = [1,2,3,4]

            for sigma in sigmas:

                n_winners, winner_loser_ratio, resection_overlap_winners_median, resection_overlap_winners_mean = check_winners(sorted_players, sigma, resection_nodes, group_size)

            Subject.append(sub)
            Outcome.append(surgical_outcome)
            CM.append(measure)
            Strategy.append(summary_stat_label[i])
            Sigmas.append(sigma)

            N_winners.append(n_winners)
            Mean_overlap_ratio.append(winner_loser_ratio)
            Median_overlap_winners.append(resection_overlap_winners_median)
            Mean_overlap_winners.append(resection_overlap_winners_mean)

surgical_outcome_prediciton_data = pd.DataFrame({"Subject":Subject, "Outcome":Outcome, "CM":CM, "Strategy":Strategy, "Sigmas":Sigmas, "N_winners":N_winners, "Mean_overlap_ratio":Mean_overlap_ratio, "Median_overlap_winners":Median_overlap_winners, "Mean_overlap_winners":Mean_overlap_winners})
surgical_outcome_prediciton_data.to_excel(path_game_scores+f"surgical_outcome_data_{sigmas}sigma.xlsx")